In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:

#libararies
import pandas as pd


In [ ]:
drive.mount('/content/gdrive')
path='/content/gdrive/My Drive/GP | Smart Meter | CIC/Datasets/Smart meters in London'

In [ ]:
for num in range(0,112):
  df = pd.read_csv(path + "/daily_dataset/daily_dataset/block_" + str(num) + ".csv")
  df = df[['day','LCLid','energy_sum']]
  df.reset_index()
  df.to_csv("hc_"+str(num)+".csv")

fout= open("energy.csv","a")
# first file:
for line in open("hc_0.csv"):
  fout.write(line)
# now the rest:    
for num in range(0,112):
  f = open("hc_"+str(num)+".csv")
  f.readline() # skip the header
  for line in f:
      fout.write(line)
  f.close()
fout.close()

In [ ]:
energy = pd.read_csv('energy.csv')
len(energy)

In [ ]:
energy.shape  

In [ ]:
#energy.head()

In [ ]:
energy.drop('Unnamed: 0',axis='columns', inplace=True)

In [ ]:
#energy.head()

In [ ]:
#energy.shape

In [ ]:
energy.day = pd.to_datetime(energy.day,format='%Y-%m-%d').dt.date

In [ ]:
energy.head()

In [ ]:
energy.shape

In [ ]:
#housecount = energy.groupby('day')[['LCLid']].nunique()
#housecount.head(10)

In [ ]:
#housecount.plot(figsize=(25,5))


# Processing over weather file

In [ ]:
weather = pd.read_csv(path + "/weather_daily_darksky.csv")
weather.head(2)

In [ ]:
#weather.describe()

In [ ]:
weather.precipType.unique()

In [ ]:
weather["precipType"].replace({"rain": 1, "snow": 0}, inplace=True)

In [ ]:
#weather.head()

In [ ]:
weather.icon.unique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
icon_one_hot_decoded = pd.get_dummies(weather.icon,prefix='icon')

In [ ]:
icon_one_hot_decoded.head()

In [ ]:
weather = pd.concat([weather, icon_one_hot_decoded], axis=1)


In [ ]:
weather.head()

In [ ]:
weather.summary.unique()

In [ ]:
weather['day']=  pd.to_datetime(weather['time']) # day is given as timestamp


In [ ]:
weather.head()

In [ ]:
weather['day']=  pd.to_datetime(weather['day'],format='%Y%m%d').dt.date
weather.head(2)

In [ ]:
# selecting numeric variables
weather = weather[['temperatureMax', 'windBearing', 'dewPoint', 'cloudCover', 'windSpeed',
       'pressure', 'apparentTemperatureHigh','precipType', 'visibility', 'humidity',
       'apparentTemperatureLow', 'apparentTemperatureMax', 'uvIndex',
       'temperatureLow', 'temperatureMin', 'temperatureHigh',
       'apparentTemperatureMin', 'moonPhase','icon_clear-day','icon_cloudy','icon_fog','icon_partly-cloudy-day','icon_partly-cloudy-night','icon_wind','day']]

In [ ]:
weather = weather.dropna()


In [ ]:
weather.head()

In [ ]:
weather.shape

#Merge Energy Dataframe with weather Dataframe

In [ ]:
weather_energy =  energy.merge(weather,on='day')
weather_energy.head(2)

In [ ]:
weather_energy.shape

Note:<br>
data = 3536007 - 3526707 <br>
lost 9300

#Household Data

In [ ]:
informations_households = pd.read_csv(path + "/informations_households.csv")
informations_households.head(20)

In [ ]:
informations_households.stdorToU.unique()

In [ ]:
informations_households["stdorToU"].replace({"ToU": 1, "Std": 0}, inplace=True)

In [ ]:
informations_households.Acorn.unique() #first trial without it without it

In [ ]:
informations_households.Acorn_grouped.unique()

In [ ]:
#Acorn_one_hot_decoded = pd.get_dummies(informations_households.Acorn_grouped,prefix='Acorn')

In [ ]:
#Acorn_one_hot_decoded.head()

#Try Label Encoding 

In [ ]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
informations_households['Acorn_grouped'] = label_encoder.fit_transform(informations_households['Acorn_grouped'])
informations_households['file'] = label_encoder.fit_transform(informations_households['file'])


In [ ]:
informations_households.head()

In [ ]:
#informations_households = pd.concat([informations_households, Acorn_one_hot_decoded], axis=1)


In [ ]:
#informations_households = informations_households.rename(columns={"file": "block"})


In [ ]:
#informations_households.block.unique()

In [ ]:
#block_one_hot_decoded = pd.get_dummies(informations_households.block)

In [ ]:
#block_one_hot_decoded.head()

In [ ]:
#informations_households['block'].astype('category')

In [ ]:
#informations_households['block'] = informations_households['block'].cat.codes

In [ ]:
#informations_households = pd.concat([informations_households, block_one_hot_decoded], axis=1)


In [ ]:
#informations_households.head()

In [ ]:
informations_households = informations_households.drop(['Acorn'], axis=1)


#Merge household data to dataset

In [ ]:
data =  weather_energy.merge(informations_households,on='LCLid')


In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
#weather.shape

In [ ]:
#weather.head()

# Holiday Data


In [ ]:
holiday = pd.read_csv(path + "/uk_bank_holidays.csv")


In [ ]:
holiday.head()

In [ ]:
#uk_bank_holidays = uk_bank_holidays.rename(columns={"Bank holidays": "day"})
holiday['Bank holidays'] = pd.to_datetime(holiday['Bank holidays'],format='%Y-%m-%d').dt.date

In [ ]:
holiday.head(4)


In [ ]:
#uk_bank_holidays.day = pd.to_datetime(uk_bank_holidays.day ,format='%Y-%m-%d').dt.date

In [ ]:
#uk_bank_holidays.day.unique()

In [ ]:
#uk_bank_holidays['Type'] = 1

In [ ]:
#uk_bank_holidays.head()

In [ ]:
#uk_bank_holidays.shape

#Merge holidays

In [ ]:
import numpy as np
data = data.merge(holiday, left_on = 'day',right_on = 'Bank holidays',how = 'left')
data['holiday_ind'] = np.where(data['Bank holidays'].isna(),0,1)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.holiday_ind.unique()

In [ ]:
#if data['day'].values == uk_bank_holidays['day'].values:
#  data['holiday'] = 1
#else:
#  data['holiday'] = 0

In [ ]:
data = data.drop(['Bank holidays','Type'], axis = 1)

In [ ]:
data.head()

#Create Last day's consumption

In [ ]:
data['last_day_sum'] = data['energy_sum'].shift()
data.head()

In [ ]:
data = data.dropna()


In [ ]:
data.shape

**After** removing nan values data lost 57 raws total of 93057

#Add Season Column

In [ ]:
data['day'] = pd.to_datetime(data.day, format='%Y-%m-%d %H:%M:%S')


In [ ]:
data['month'] = data['day'].dt.month 


In [ ]:
data.head()

In [ ]:
data.loc[(data['month'] == 12) | (data['month'] == 1) | (data['month'] == 2) , 'season'] = 'winter'
data.loc[(data['month'] == 3) | (data['month'] == 4) | (data['month'] == 5) , 'season'] = 'spring'
data.loc[(data['month'] == 6) | (data['month'] == 7) | (data['month'] == 8) , 'season'] = 'summer'
data.loc[(data['month'] == 9) | (data['month'] == 10) | (data['month'] ==11) , 'season'] = 'autumn'

In [ ]:
data.head()

In [ ]:
data.season.unique()

In [ ]:
data['season'] = label_encoder.fit_transform(data['season'])


In [ ]:
data.head()

In [ ]:
data.season.unique()

In [ ]:
data.shape

#First without any feature scaling or Validation 


## Split data to traing and testing

In [ ]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, 3:].values
y = data.iloc[:,2].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=9)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#regressor = RandomForestRegressor(n_estimators=8000,random_state= 42)
regressor = RandomForestRegressor(n_estimators=1000,random_state= 42)

regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)


In [ ]:
rrmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred)) / np.mean(y_test)
print('Relative root mean squared error percentage:', rrmse*100)


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

regressor2 = ExtraTreesRegressor(n_estimators=8000, random_state=42)
regressor2.fit(X_train, y_train)
y_pred2 = regressor2.predict(X_test)

In [ ]:
rrmse2 = np.sqrt(metrics.mean_squared_error(y_test, y_pred2)) / np.mean(y_test)
print('Relative root mean squared error percentage:', rrmse2*100)